<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Options_Trading_Strategy_based_on_ARIMA_Forecasting_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ibapi

from ibapi.client import EClient
from ibapi.contract import Contract
from ibapi.order import *
from ibapi.wrapper import EWrapper

class TradingApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.nextOrderId = None  # Initialize nextOrderId as None
        self.data = {}  # Store option data
        self.positions = {}  # Store open positions

    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextOrderId = orderId

    def contractDetails(self, reqId: int, contractDetails):
        super().contractDetails(reqId, contractDetails)
        contract = contractDetails.contract
        symbol = contract.symbol
        right = contract.right
        self.data[symbol] = {"symbol": symbol, "right": right, "strike": contract.strike}

    def tickPrice(self, reqId, tickType, price, attrib):
        super().tickPrice(reqId, tickType, price, attrib)
        if tickType == 1:  # Last price
            symbol = self.data[reqId]["symbol"]
            right = self.data[reqId]["right"]
            strike = self.data[reqId]["strike"]
            print(f"Received last price for {symbol}: {price}")

            # Implement your buy and sell order logic here
            if price > threshold:
                # Check if position is already open for the symbol
                if symbol in self.positions and self.positions[symbol] > 0:
                    print(f"Position for {symbol} already open. Skipping buy order.")
                else:
                    self.placeOrder(symbol, right, strike, "BUY", quantity)
            elif price < threshold:
                # Check if position is already open for the symbol
                if symbol in self.positions and self.positions[symbol] < 0:
                    print(f"Position for {symbol} already open. Skipping sell order.")
                else:
                    self.placeOrder(symbol, right, strike, "SELL", quantity)

    def tickOptionComputation(self, reqId, tickType, tickValue):
        super().tickOptionComputation(reqId, tickType, tickValue)
        if tickType == 10:  # Model option computation
            symbol = self.data[reqId]["symbol"]
            impliedVolatility = tickValue.impliedVol
            delta = tickValue.delta
            gamma = tickValue.gamma
            # Implement your logic using the option Greeks and other values
            print(f"Received option computation for {symbol}: IV={impliedVolatility}, Delta={delta}, Gamma={gamma}")

    def placeOrder(self, symbol, right, strike, action, quantity):
        contract = Contract()
        contract.symbol = symbol
        contract.secType = "OPT"
        contract.exchange = "SMART"
        contract.currency = "USD"
        contract.lastTradeDateOrContractMonth = "20231215"
        contract.strike = strike
        contract.right = right
        contract.multiplier = "100"

        order = Order()
        order.action = action
        order.totalQuantity = quantity
        order.orderType = "LMT"
        order.lmtPrice = limit_price

        self.placeOrderWithContract(self.nextOrderId, contract, order)  # Update method name
        self.nextOrderId += 1

    def error(self, reqId, errorCode, errorString):
        super().error(reqId, errorCode, errorString)
        print(f"Error: {errorCode}, {errorString}")

    def contractDetailsEnd(self, reqId):
        super().contractDetailsEnd(reqId)
        print("Contract details request completed.")

    def position(self, account, contract, position, avgCost):
        super().position(account, contract, position, avgCost)
        symbol = contract.symbol
        self.positions[symbol] = position
        print(f"Received position update for {symbol}: {position}")

    def positionEnd(self):
        super().positionEnd()
        print("Position request completed.")

app = TradingApp()
app.connect("127.0.0.1", 4002, clientId=1)

# Define your options contract details
symbols = ["AAPL", "TSLA", "GOOGL"]
quantity = 1
threshold = 1.0  # Define your threshold value
limit_price = 0.0  # Define your limit price

app.reqPositions()  # Request open positions

# Request market data for the options contracts
for symbol in symbols:
    contract = Contract()
    contract.symbol = symbol
    contract.secType = "OPT"
    contract.exchange = "SMART"
    contract.currency = "USD"
    contract.lastTradeDateOrContractMonth = "20231215"
    contract.strike = 150
    contract.right = "C"
    contract.multiplier = "100"

    app.reqMarketDataType(4)
    app.reqContractDetails(len(app.data), contract)
    app.reqMktData(len(app.data), contract, "", False, False, [])
    app.nextOrderId += 1

app.run()
